In [1]:
import pandas as pd
import json

In [2]:
main_df = pd.read_csv("data/db_main.csv")

state_pop = pd.read_csv("data/pop_usa.csv")

In [3]:
main_df

,id,address,categories,city,country,keys,latitude,longitude,menuPageURL,menus.amountMax,...,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,state
0,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,22.50,...,2016-03-31T02:19:42Z,NaN,Bianca Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR,Oregon
1,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,18.95,...,2016-03-31T02:19:42Z,NaN,Cheese Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR,Oregon
2,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,12.00,...,2015-10-23T03:57:26Z,NaN,"Pizza, Margherita",The Brentwood,90049,USD,50.0,55.0,Brentwood,California
3,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,2015-10-23T03:57:26Z,NaN,"Pizza, Mushroom",The Brentwood,90049,USD,50.0,55.0,Brentwood,California
4,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,2015-10-23T03:57:26Z,"Olives, onions, capers, tomatoes","Pizza, Puttenesca",The Brentwood,90049,USD,50.0,55.0,Brentwood,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,AVweYUZ0IN2L1WUf4Z9q,305 Ash St,Bar and Gastropub,Jefferson City,US,us/honeycreek/jeffersoncity/305ashst/-735513695,38.568717,-92.161596,NaN,11.99,...,2016-07-06T20:05:49Z,NaN,Supreme Pizza,Prison Brews Brewery & Restaurant,65101,USD,25.0,40.0,Honey Creek,Missouri
3506,AVweYUZ0IN2L1WUf4Z9q,305 Ash St,Bar and Gastropub,Jefferson City,US,us/honeycreek/jeffersoncity/305ashst/-735513695,38.568717,-92.161596,NaN,9.99,...,2016-07-06T20:05:49Z,NaN,Vegetarian Pizza,Prison Brews Brewery & Restaurant,65101,USD,25.0,40.0,Honey Creek,Missouri
3507,AVweZ5SbIN2L1WUf4nWU,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,us/davidsburg/dover/4140carlislerd/-1821521454,39.996444,-76.845180,NaN,5.00,...,2015-10-23T01:03:47Z,NaN,Pita Pizza,Moonlight Cafe,17315,USD,0.0,30.0,Davidsburg,Pennsylvania
3508,AVweZ5SbIN2L1WUf4nWU,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,us/davidsburg/dover/4140carlislerd/-1821521454,39.996444,-76.845180,NaN,20.00,...,2015-10-23T01:03:47Z,NaN,Steak Pizzaiola,Moonlight Cafe,17315,USD,0.0,30.0,Davidsburg,Pennsylvania


In [5]:
main_df["coords"] = main_df.apply(lambda x : [x.latitude, x.longitude], axis=1)

resto_by_state = main_df[["id", "state", "address"]].drop_duplicates(subset=["address"]).groupby("state").count().rename(columns={"id":"count"}).reset_index()
restos_num_by_state = dict(zip(resto_by_state.state, resto_by_state["count"]))

with_pop = resto_by_state.merge(state_pop, on="state")
with_pop["restos_by_pop"] = (with_pop["count"]*1000000)/with_pop["pop"]
restos_by_pop = dict(zip(with_pop.state, with_pop["restos_by_pop"]))

restos_coords = main_df.groupby('state')['coords'].apply(list).reset_index()
restos_coords = dict(zip(restos_coords.state, restos_coords["coords"]))

In [11]:
restos_num_by_state

{'Alabama': 5,
 'Alaska': 1,
 'Arizona': 18,
 'Arkansas': 5,
 'California': 89,
 'Colorado': 20,
 'Connecticut': 6,
 'Delaware': 4,
 'District of Columbia': 3,
 'Florida': 51,
 'Georgia': 15,
 'Hawaii': 5,
 'Idaho': 3,
 'Illinois': 58,
 'Indiana': 17,
 'Iowa': 6,
 'Kansas': 5,
 'Kentucky': 7,
 'Louisiana': 9,
 'Maine': 7,
 'Maryland': 22,
 'Massachusetts': 41,
 'Michigan': 24,
 'Minnesota': 19,
 'Mississippi': 5,
 'Missouri': 12,
 'Montana': 5,
 'Nebraska': 6,
 'Nevada': 9,
 'New Hampshire': 4,
 'New Jersey': 74,
 'New Mexico': 3,
 'New York': 101,
 'North Carolina': 25,
 'North Dakota': 1,
 'Ohio': 38,
 'Oklahoma': 7,
 'Oregon': 15,
 'Pennsylvania': 65,
 'Puerto Rico': 1,
 'Rhode Island': 11,
 'South Carolina': 14,
 'South Dakota': 1,
 'Tennessee': 9,
 'Texas': 51,
 'Utah': 10,
 'Vermont': 3,
 'Virginia': 15,
 'Washington': 26,
 'West Virginia': 5,
 'Wisconsin': 10,
 'Wyoming': 1}

In [9]:
def write_json_file(to_add, new_feature_name):
    f = open("data/map-data.json", "r")
    y = json.load(f)
    for elem in y["features"]:
        elem_to_add = elem["properties"]["name"]
        if elem_to_add != "Puerto Rico":
            elem["properties"][new_feature_name] = to_add[elem_to_add]
    f.close()
    f = open("data/map-data.json", "w")
    json.dump(y, f)
    f.close()

In [10]:
write_json_file(restos_num_by_state, "num_of_restos")
write_json_file(restos_by_pop, "restos_by_pop")
write_json_file(restos_coords, "restos_coords")


In [91]:
new.loc[new.state=="New York"]

,state,count,pop,lat,long,restos_by_pop
32,New York,374,19453561,42.652843,-73.757874,19.225272
